In [32]:
import pandas as pd
import numpy as np
import zipfile
from scipy import stats

In [3]:
with zipfile.ZipFile('child-mind-institute-problematic-internet-use.zip') as z:
    with z.open('series_train.parquet/id=00f332d1/part-0.parquet') as f:
        #df = pd.read_csv(f)
        df_obs = pd.read_parquet(f)

df_obs    # time series data for one of the observations

,step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,time_of_day,weekday,quarter,relative_date_PCIAT
0,0,-0.051997,0.011325,-0.991420,0.026643,-86.152954,0.0,30.000000,4185.000000,57240000000000,1,2,14.0
1,1,-0.008801,-0.036264,-1.000591,0.001868,-87.851646,0.0,30.000000,4185.000000,57245000000000,1,2,14.0
2,2,-0.008904,-0.035101,-1.000357,0.001776,-87.853790,0.0,30.333334,4185.000000,57250000000000,1,2,14.0
3,3,-0.009110,-0.035204,-1.000435,0.001675,-87.850815,0.0,30.666666,4185.000000,57255000000000,1,2,14.0
4,4,-0.009020,-0.035204,-1.000709,0.001846,-87.850815,0.0,31.000000,4185.000000,57260000000000,1,2,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
414379,414379,0.015927,-0.021750,-0.985415,0.000000,-88.438774,1.0,0.500000,3098.833252,55535000000000,4,2,38.0
414380,414380,0.016043,-0.021879,-0.985102,0.000000,-88.445999,1.0,0.500000,3098.666748,55540000000000,4,2,38.0
414381,414381,0.015888,-0.021814,-0.985428,0.000000,-88.451225,1.0,0.500000,3098.500000,55545000000000,4,2,38.0
414382,414382,0.016352,-0.021762,-0.985741,0.000000,-88.456245,1.0,0.500000,3098.333252,55550000000000,4,2,38.0


In [258]:
df_desc = pd.read_csv('content/data_dictionary.csv')
df_desc

,Instrument,Field,Description,Type,Values,Value Labels
0,Identifier,id,Participant's ID,str,NaN,NaN
1,Demographics,Basic_Demos-Enroll_Season,Season of enrollment,str,"Spring, Summer, Fall, Winter",NaN
2,Demographics,Basic_Demos-Age,Age of participant,float,NaN,NaN
3,Demographics,Basic_Demos-Sex,Sex of participant,categorical int,"0,1","0=Male, 1=Female"
4,Children's Global Assessment Scale,CGAS-Season,Season of participation,str,"Spring, Summer, Fall, Winter",NaN
...,...,...,...,...,...,...
76,Sleep Disturbance Scale,SDS-Season,Season of participation,str,"Spring, Summer, Fall, Winter",NaN
77,Sleep Disturbance Scale,SDS-SDS_Total_Raw,Total Raw Score,int,NaN,NaN
78,Sleep Disturbance Scale,SDS-SDS_Total_T,Total T-Score,int,NaN,NaN
79,Internet Use,PreInt_EduHx-Season,Season of participation,str,"Spring, Summer, Fall, Winter",NaN


In [280]:
df = pd.read_csv('content/train.csv')
df

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,ff8a2de4,Fall,13,0,Spring,60.0,Fall,16.362460,59.5,82.4,...,1.0,1.0,0.0,32.0,Winter,35.0,50.0,Fall,1.0,1.0
3956,ffa9794a,Winter,10,0,NaN,NaN,Spring,18.764678,53.5,76.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winter,0.0,NaN
3957,ffcd4dbd,Fall,11,0,Spring,68.0,Winter,21.441500,60.0,109.8,...,1.0,0.0,1.0,31.0,Winter,56.0,77.0,Fall,0.0,1.0
3958,ffed1dd5,Spring,13,0,Spring,70.0,Winter,12.235895,70.7,87.0,...,1.0,1.0,1.0,19.0,Spring,33.0,47.0,Spring,1.0,0.0


In [6]:
df.shape

(3960, 82)

# Before we begin...

Our data has categorical information, true numerical information, and categorical information stored as numbers

In [225]:
df_desc.head() # look at type column

,Instrument,Field,Description,Type,Values,Value Labels
0,Identifier,id,Participant's ID,str,NaN,NaN
1,Demographics,Basic_Demos-Enroll_Season,Season of enrollment,str,"Spring, Summer, Fall, Winter",NaN
2,Demographics,Basic_Demos-Age,Age of participant,float,NaN,NaN
3,Demographics,Basic_Demos-Sex,Sex of participant,categorical int,"0,1","0=Male, 1=Female"
4,Children's Global Assessment Scale,CGAS-Season,Season of participation,str,"Spring, Summer, Fall, Winter",NaN


In [226]:
df_desc['Type'].unique()

array(['str', 'float', 'categorical int', 'int'], dtype=object)

In [257]:
df_desc[df_desc['Field'] == 'sii'] # worthy to note that the description table doesn't have a row for the output sii

,Instrument,Field,Description,Type,Values,Value Labels


In [263]:
list(df_desc[df_desc['Field'] == 'PCIAT-PCIAT_Total']['Value Labels'])

['Severity Impairment Index: 0-30=None; 31-49=Mild; 50-79=Moderate; 80-100=Severe']

In [281]:
# categoricals = df.dtypes[df.dtypes == 'object'].index
categoricals = pd.Index(df_desc[df_desc['Type'] == 'str']['Field'])
categoricals # season data + id

Index(['id', 'Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season',
       'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 'PAQ_A-Season',
       'PAQ_C-Season', 'PCIAT-Season', 'SDS-Season', 'PreInt_EduHx-Season'],
      dtype='object', name='Field')

In [282]:
# allnumericals = df.dtypes[df.dtypes != 'object'].index
allnumericals = pd.Index(df_desc[df_desc['Type'] != 'str']['Field'])
allnumericals

Index(['Basic_Demos-Age', 'Basic_Demos-Sex', 'CGAS-CGAS_Score', 'Physical-BMI',
       'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
       'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
       'Fitness_Endurance-Max_Stage', 'Fitness_Endurance-Time_Mins',
       'Fitness_Endurance-Time_Sec', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone',
       'FGC-FGC_GSND', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone',
       'FGC-FGC_PU', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone',
       'FGC-FGC_SRR', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone',
       'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
       'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
       'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
       'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
       'BIA-BIA_TBW', 'PAQ_A-PAQ_A_Total', 'PAQ_C-PAQ_C_Total',
       'PCIAT-PCIAT_01', 'PCIAT-PCIAT_02', 'PCIAT-PCIAT_03

In [283]:
# numcategoricals = allnumericals[(df[allnumericals].min() == 0) & (df[allnumericals].max() <= 5)]
numcategoricals = pd.Index(df_desc[df_desc['Type'] == 'categorical int']['Field']).append(pd.Index(['sii'],name='Field'))
numcategoricals

Index(['Basic_Demos-Sex', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND_Zone',
       'FGC-FGC_GSD_Zone', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL_Zone',
       'FGC-FGC_SRR_Zone', 'FGC-FGC_TL_Zone', 'BIA-BIA_Activity_Level_num',
       'BIA-BIA_Frame_num', 'PCIAT-PCIAT_01', 'PCIAT-PCIAT_02',
       'PCIAT-PCIAT_03', 'PCIAT-PCIAT_04', 'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06',
       'PCIAT-PCIAT_07', 'PCIAT-PCIAT_08', 'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10',
       'PCIAT-PCIAT_11', 'PCIAT-PCIAT_12', 'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14',
       'PCIAT-PCIAT_15', 'PCIAT-PCIAT_16', 'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18',
       'PCIAT-PCIAT_19', 'PCIAT-PCIAT_20',
       'PreInt_EduHx-computerinternet_hoursday', 'sii'],
      dtype='object', name='Field')

In [284]:
# numericals = allnumericals[~((df[allnumericals].min() == 0) & (df[allnumericals].max() <= 5))]
numericals = pd.Index(df_desc[(df_desc['Type'] == 'float') | (df_desc['Type'] == 'int')]['Field'])
numericals

Index(['Basic_Demos-Age', 'CGAS-CGAS_Score', 'Physical-BMI', 'Physical-Height',
       'Physical-Weight', 'Physical-Waist_Circumference',
       'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
       'Fitness_Endurance-Max_Stage', 'Fitness_Endurance-Time_Mins',
       'Fitness_Endurance-Time_Sec', 'FGC-FGC_CU', 'FGC-FGC_GSND',
       'FGC-FGC_GSD', 'FGC-FGC_PU', 'FGC-FGC_SRL', 'FGC-FGC_SRR', 'FGC-FGC_TL',
       'BIA-BIA_BMC', 'BIA-BIA_BMI', 'BIA-BIA_BMR', 'BIA-BIA_DEE',
       'BIA-BIA_ECW', 'BIA-BIA_FFM', 'BIA-BIA_FFMI', 'BIA-BIA_FMI',
       'BIA-BIA_Fat', 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST',
       'BIA-BIA_SMM', 'BIA-BIA_TBW', 'PAQ_A-PAQ_A_Total', 'PAQ_C-PAQ_C_Total',
       'PCIAT-PCIAT_Total', 'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T'],
      dtype='object', name='Field')

In [285]:
len(categoricals) + len(numcategoricals) + len(numericals), df.shape[1]

(82, 82)

# Cleaning up the Zeros

Some of the data is filled in with 0 when it really should be `NaN`

In [7]:
df.describe().loc['min'][df.describe().loc['min'] < 0].index

Index(['BIA-BIA_BMC', 'BIA-BIA_FMI', 'BIA-BIA_Fat'], dtype='object')

These columns have a negative minimum. Is this possible?

In [8]:
df_desc[df_desc['Field'].isin(df.describe().loc['min'][df.describe().loc['min'] < 0].index)]

,Instrument,Field,Description,Type,Values,Value Labels
35,Bio-electric Impedance Analysis,BIA-BIA_BMC,Bone Mineral Content,float,NaN,NaN
42,Bio-electric Impedance Analysis,BIA-BIA_FMI,Fat Mass Index,float,NaN,NaN
43,Bio-electric Impedance Analysis,BIA-BIA_Fat,Body Fat Percentage,float,NaN,NaN


You cannot have negative bone mineral content, negative fat mass index, or negative body fat percentage. We should replace the negative values with `NaN`.

In [9]:
replaceNegatives = list(df.describe().loc['min'][df.describe().loc['min'] < 0].index)
replaceNegatives

['BIA-BIA_BMC', 'BIA-BIA_FMI', 'BIA-BIA_Fat']

In [10]:
df.describe().loc['min'][df.describe().loc['min'] == 0].index

Index(['Basic_Demos-Sex', 'Physical-BMI', 'Physical-Weight',
       'Physical-Diastolic_BP', 'Physical-Systolic_BP',
       'Fitness_Endurance-Max_Stage', 'Fitness_Endurance-Time_Mins',
       'Fitness_Endurance-Time_Sec', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone',
       'FGC-FGC_GSND', 'FGC-FGC_GSD', 'FGC-FGC_PU', 'FGC-FGC_PU_Zone',
       'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR', 'FGC-FGC_SRR_Zone',
       'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'PCIAT-PCIAT_01', 'PCIAT-PCIAT_02',
       'PCIAT-PCIAT_03', 'PCIAT-PCIAT_04', 'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06',
       'PCIAT-PCIAT_07', 'PCIAT-PCIAT_08', 'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10',
       'PCIAT-PCIAT_11', 'PCIAT-PCIAT_12', 'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14',
       'PCIAT-PCIAT_15', 'PCIAT-PCIAT_16', 'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18',
       'PCIAT-PCIAT_19', 'PCIAT-PCIAT_20', 'PCIAT-PCIAT_Total',
       'PreInt_EduHx-computerinternet_hoursday', 'sii'],
      dtype='object')

In [287]:
numericals.intersection(df.describe().loc['min'][df.describe().loc['min'] == 0].index)

Index(['Physical-BMI', 'Physical-Weight', 'Physical-Diastolic_BP',
       'Physical-Systolic_BP', 'Fitness_Endurance-Max_Stage',
       'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
       'FGC-FGC_CU', 'FGC-FGC_GSND', 'FGC-FGC_GSD', 'FGC-FGC_PU',
       'FGC-FGC_SRL', 'FGC-FGC_SRR', 'FGC-FGC_TL', 'PCIAT-PCIAT_Total'],
      dtype='object')

In [288]:
numcategoricals.intersection(df.describe().loc['min'][df.describe().loc['min'] == 0].index)

Index(['Basic_Demos-Sex', 'FGC-FGC_CU_Zone', 'FGC-FGC_PU_Zone',
       'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL_Zone',
       'PCIAT-PCIAT_01', 'PCIAT-PCIAT_02', 'PCIAT-PCIAT_03', 'PCIAT-PCIAT_04',
       'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07', 'PCIAT-PCIAT_08',
       'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10', 'PCIAT-PCIAT_11', 'PCIAT-PCIAT_12',
       'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14', 'PCIAT-PCIAT_15', 'PCIAT-PCIAT_16',
       'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18', 'PCIAT-PCIAT_19', 'PCIAT-PCIAT_20',
       'PreInt_EduHx-computerinternet_hoursday', 'sii'],
      dtype='object')

These columns have a minimum of 0. Is this possible?

In [11]:
df_desc[df_desc['Field'].isin(df.describe().loc['min'][df.describe().loc['min'] == 0].index)]

,Instrument,Field,Description,Type,Values,Value Labels
3,Demographics,Basic_Demos-Sex,Sex of participant,categorical int,"0,1","0=Male, 1=Female"
7,Physical Measures,Physical-BMI,Body Mass Index (kg/m^2),float,NaN,NaN
9,Physical Measures,Physical-Weight,Weight (lbs),float,NaN,NaN
11,Physical Measures,Physical-Diastolic_BP,Diastolic BP (mmHg),int,NaN,NaN
13,Physical Measures,Physical-Systolic_BP,Systolic BP (mmHg),int,NaN,NaN
15,FitnessGram Vitals and Treadmill,Fitness_Endurance-Max_Stage,Maximum stage reached,int,NaN,NaN
16,FitnessGram Vitals and Treadmill,Fitness_Endurance-Time_Mins,Exact time completed: Minutes,int,NaN,NaN
17,FitnessGram Vitals and Treadmill,Fitness_Endurance-Time_Sec,Exact time completed: Seconds,int,NaN,NaN
19,FitnessGram Child,FGC-FGC_CU,Curl up total,int,NaN,NaN
20,FitnessGram Child,FGC-FGC_CU_Zone,Curl up fitness zone,categorical int,"0,1","0=Needs Improvement, 1=Healthy Fitness Zone"


A value of 0 is impossible for BMI, weight, blood pressure, or any fitness test that counts reps.

We should replace these 0 values with `NaN`.

In [12]:
replaceZeros = ['FGC-FGC_TL', 'FGC-FGC_SRR', 'FGC-FGC_SRL', 'FGC-FGC_PU','FGC-FGC_GSD','FGC-FGC_GSND','FGC-FGC_CU','Fitness_Endurance-Time_Sec', 'Fitness_Endurance-Time_Mins','Fitness_Endurance-Max_Stage', 'Physical-Systolic_BP', 'Physical-Diastolic_BP','Physical-Weight','Physical-BMI']
replaceZeros.reverse()

In [13]:
replaceZeros

['Physical-BMI',
 'Physical-Weight',
 'Physical-Diastolic_BP',
 'Physical-Systolic_BP',
 'Fitness_Endurance-Max_Stage',
 'Fitness_Endurance-Time_Mins',
 'Fitness_Endurance-Time_Sec',
 'FGC-FGC_CU',
 'FGC-FGC_GSND',
 'FGC-FGC_GSD',
 'FGC-FGC_PU',
 'FGC-FGC_SRL',
 'FGC-FGC_SRR',
 'FGC-FGC_TL']

In [296]:
df[df['PCIAT-PCIAT_Total'] == 0]['Basic_Demos-Age'].max()

17

In [14]:
df[replaceNegatives] = df[replaceNegatives].mask(df[replaceNegatives] <= 0)

In [15]:
(df[replaceNegatives] <= 0).sum().sum()

0

In [16]:
df[replaceZeros] = df[replaceZeros].mask(df[replaceZeros] <= 0)

In [17]:
(df[replaceZeros] <= 0).sum().sum()

0

In [18]:
df.isnull().sum().sum() / (df.shape[0] * df.shape[1])

0.4108770633160877

41% of the data is null

In [81]:
# df[(df[df.dtypes.index[df.dtypes != 'object']] < 0).any(axis=1)]

In [82]:
# df[df.dtypes.index[df.dtypes != 'object']] < 0

In [83]:
# df.mask(df[df.dtypes.index[df.dtypes != 'object']] < 0, np.nan)
# df[df.dtypes.index[df.dtypes != 'object']].mask(df[df.dtypes.index[df.dtypes != 'object']] < 0, np.nan)

## Removing columns in which more than 80% of the rows are null

In [19]:
(df.isnull().sum() > len(df)*0.8).sum()

5

There are 5 columns who have >80% missing values.

In [20]:
list(df.columns[df.isnull().sum() > len(df)*0.8])

['Fitness_Endurance-Max_Stage',
 'Fitness_Endurance-Time_Mins',
 'Fitness_Endurance-Time_Sec',
 'PAQ_A-Season',
 'PAQ_A-PAQ_A_Total']

In [21]:
df_desc[df_desc['Field'].isin(df.columns[df.isnull().sum() > len(df)*0.8])]

,Instrument,Field,Description,Type,Values,Value Labels
15,FitnessGram Vitals and Treadmill,Fitness_Endurance-Max_Stage,Maximum stage reached,int,NaN,NaN
16,FitnessGram Vitals and Treadmill,Fitness_Endurance-Time_Mins,Exact time completed: Minutes,int,NaN,NaN
17,FitnessGram Vitals and Treadmill,Fitness_Endurance-Time_Sec,Exact time completed: Seconds,int,NaN,NaN
50,Physical Activity Questionnaire (Adolescents),PAQ_A-Season,Season of participation,str,"Spring, Summer, Fall, Winter",NaN
51,Physical Activity Questionnaire (Adolescents),PAQ_A-PAQ_A_Total,Activity Summary Score (Adolescents),float,NaN,NaN


In [22]:
"""
filter_null = df.isnull().sum().sort_values(ascending = True) <= (len(df) / 2)
filter_null = filter_null[filter_null == True]
list_of_columns_less_null = filter_null.index.to_list()
df_filter_null = df[list_of_columns_less_null]
df_filter_null.isna().sum().sum() / len(df_filter_null)
len(df_filter_null)
"""
df = df.drop(columns=df.columns[df.isnull().sum() > len(df)*0.8])
df.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df.isnull().sum().sum() / (df.shape[0] * df.shape[1])

0.38279876688967596

38% of the data is now null.

In [24]:
(df.isnull().sum(axis=1) > df.shape[1]*0.8).sum()

756

756 rows of the data have more than 80% of the attributes missing! Consider removing those rows.  
On the other hand, we also have time series data as well for some of the rows, so we can wait and see later on if we really need to remove the rows that have a lot of missing values.

# Removing the outliers

In [31]:
df.describe()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,...,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,sii
count,3960.000000,3960.000000,2421.000000,3015.000000,3027.000000,3015.000000,898.000000,2953.000000,2967.000000,2953.000000,...,2728.000000,2725.000000,2728.000000,2730.000000,2733.000000,2736.000000,2609.000000,2606.000000,3301.000000,2736.000000
mean,10.433586,0.372727,65.454771,19.376812,55.946713,90.840060,27.278508,69.672536,81.597236,117.022689,...,1.452346,1.627890,1.613636,1.158974,0.943652,27.896199,41.088923,57.763622,1.060588,0.580409
std,3.574648,0.483591,22.341862,5.034191,7.473764,43.161374,5.567287,13.553022,13.665196,16.927687,...,1.495600,1.445622,1.529178,1.343661,1.185460,20.338853,10.427433,13.196091,1.094875,0.771122
min,5.000000,0.000000,25.000000,8.522436,33.000000,31.800000,18.000000,11.000000,27.000000,49.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17.000000,38.000000,0.000000,0.000000
25%,8.000000,0.000000,59.000000,15.890526,50.000000,58.200000,23.000000,61.000000,72.000000,107.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,33.000000,47.000000,0.000000,0.000000
50%,10.000000,0.000000,65.000000,17.950925,55.000000,77.800000,26.000000,68.000000,81.000000,114.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,26.000000,39.000000,55.000000,1.000000,0.000000
75%,13.000000,1.000000,75.000000,21.588631,62.000000,114.300000,30.000000,76.000000,90.500000,125.000000,...,2.000000,3.000000,2.000000,2.000000,1.000000,41.000000,46.000000,64.000000,2.000000,1.000000
max,22.000000,1.000000,999.000000,59.132048,78.500000,315.000000,50.000000,179.000000,138.000000,203.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,93.000000,96.000000,100.000000,3.000000,3.000000


In [36]:
df.dtypes[df.dtypes != 'object'].unique()

array([dtype('int64'), dtype('float64')], dtype=object)

In [37]:
df.dtypes[df.dtypes == 'object']

id                           object
Basic_Demos-Enroll_Season    object
CGAS-Season                  object
Physical-Season              object
Fitness_Endurance-Season     object
FGC-Season                   object
BIA-Season                   object
PAQ_C-Season                 object
PCIAT-Season                 object
SDS-Season                   object
PreInt_EduHx-Season          object
dtype: object

The only non-numerical information is the id and the season data

In [42]:
numerical_cols = df.dtypes[df.dtypes != 'object'].index
numerical_cols

Index(['Basic_Demos-Age', 'Basic_Demos-Sex', 'CGAS-CGAS_Score', 'Physical-BMI',
       'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
       'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
       'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND', 'FGC-FGC_GSND_Zone',
       'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU', 'FGC-FGC_PU_Zone',
       'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR', 'FGC-FGC_SRR_Zone',
       'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-BIA_Activity_Level_num',
       'BIA-BIA_BMC', 'BIA-BIA_BMI', 'BIA-BIA_BMR', 'BIA-BIA_DEE',
       'BIA-BIA_ECW', 'BIA-BIA_FFM', 'BIA-BIA_FFMI', 'BIA-BIA_FMI',
       'BIA-BIA_Fat', 'BIA-BIA_Frame_num', 'BIA-BIA_ICW', 'BIA-BIA_LDM',
       'BIA-BIA_LST', 'BIA-BIA_SMM', 'BIA-BIA_TBW', 'PAQ_C-PAQ_C_Total',
       'PCIAT-PCIAT_01', 'PCIAT-PCIAT_02', 'PCIAT-PCIAT_03', 'PCIAT-PCIAT_04',
       'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07', 'PCIAT-PCIAT_08',
       'PCIAT-PCIAT_09', 'P

However! Some of the "numerical" information is actually just a scale from 0-X. For example:

In [156]:
df_desc.iloc[55:60]

,Instrument,Field,Description,Type,Values,Value Labels
55,Parent-Child Internet Addiction Test,PCIAT-PCIAT_01,How often does your child disobey time limits ...,categorical int,"0,1,2,3,4,5","0=Does Not Apply, 1=Rarely, 2=Occasionally, 3=..."
56,Parent-Child Internet Addiction Test,PCIAT-PCIAT_02,How often does your child neglect household ch...,categorical int,"0,1,2,3,4,5","0=Does Not Apply, 1=Rarely, 2=Occasionally, 3=..."
57,Parent-Child Internet Addiction Test,PCIAT-PCIAT_03,How often does your child prefer to spend time...,categorical int,"0,1,2,3,4,5","0=Does Not Apply, 1=Rarely, 2=Occasionally, 3=..."
58,Parent-Child Internet Addiction Test,PCIAT-PCIAT_04,How often does your child form new relationshi...,categorical int,"0,1,2,3,4,5","0=Does Not Apply, 1=Rarely, 2=Occasionally, 3=..."
59,Parent-Child Internet Addiction Test,PCIAT-PCIAT_05,How often do you complain about the amount of ...,categorical int,"0,1,2,3,4,5","0=Does Not Apply, 1=Rarely, 2=Occasionally, 3=..."


In [176]:
numerical_cols[(df[numerical_cols].min() == 0) & (df[numerical_cols].max() <= 5)]

Index(['Basic_Demos-Sex', 'FGC-FGC_CU_Zone', 'FGC-FGC_PU_Zone',
       'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL_Zone',
       'PCIAT-PCIAT_01', 'PCIAT-PCIAT_02', 'PCIAT-PCIAT_03', 'PCIAT-PCIAT_04',
       'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07', 'PCIAT-PCIAT_08',
       'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10', 'PCIAT-PCIAT_11', 'PCIAT-PCIAT_12',
       'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14', 'PCIAT-PCIAT_15', 'PCIAT-PCIAT_16',
       'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18', 'PCIAT-PCIAT_19', 'PCIAT-PCIAT_20',
       'PreInt_EduHx-computerinternet_hoursday', 'sii'],
      dtype='object')

In [177]:
numerical_cols[~((df[numerical_cols].min() == 0) & (df[numerical_cols].max() <= 5))]

Index(['Basic_Demos-Age', 'CGAS-CGAS_Score', 'Physical-BMI', 'Physical-Height',
       'Physical-Weight', 'Physical-Waist_Circumference',
       'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
       'FGC-FGC_CU', 'FGC-FGC_GSND', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD',
       'FGC-FGC_GSD_Zone', 'FGC-FGC_PU', 'FGC-FGC_SRL', 'FGC-FGC_SRR',
       'FGC-FGC_TL', 'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC',
       'BIA-BIA_BMI', 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW',
       'BIA-BIA_FFM', 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat',
       'BIA-BIA_Frame_num', 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST',
       'BIA-BIA_SMM', 'BIA-BIA_TBW', 'PAQ_C-PAQ_C_Total', 'PCIAT-PCIAT_Total',
       'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T'],
      dtype='object')

In [182]:
true_numericals = numerical_cols[~((df[numerical_cols].min() == 0) & (df[numerical_cols].max() <= 5))]

In [183]:
df[(np.abs(stats.zscore(df[true_numericals], nan_policy='omit')) >= 3).any(axis=1)]

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
6,0038ba98,Fall,10,0,NaN,NaN,Fall,19.660760,55.00,84.6,...,4.0,1.0,0.0,20.0,Winter,27.0,40.0,Fall,3.0,0.0
51,02cebf33,Spring,12,1,Spring,77.0,Summer,19.751301,62.00,108.0,...,3.0,3.0,3.0,46.0,Summer,82.0,100.0,Spring,1.0,1.0
59,0380e709,Summer,17,0,NaN,NaN,Summer,24.446886,68.00,160.8,...,4.0,2.0,2.0,56.0,NaN,NaN,NaN,Summer,NaN,2.0
87,04df895a,Winter,17,0,NaN,NaN,Winter,26.630764,77.00,224.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winter,2.0,NaN
193,0acac70c,Spring,18,0,Fall,65.0,Summer,21.485616,70.15,150.4,...,0.0,3.0,4.0,34.0,Summer,33.0,47.0,Spring,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3876,f9ca9ff6,Winter,11,1,NaN,NaN,Spring,16.319643,56.00,72.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winter,1.0,NaN
3903,fb9b060f,Summer,16,1,Fall,65.0,Summer,21.266964,67.00,135.8,...,3.0,2.0,1.0,52.0,Summer,68.0,93.0,Summer,2.0,2.0
3922,fcd9a6bb,Spring,6,0,Summer,69.0,Spring,16.195145,44.00,44.6,...,1.0,0.0,0.0,14.0,Spring,38.0,54.0,Spring,0.0,0.0
3946,feb53635,Winter,14,0,Summer,70.0,Spring,19.036779,69.50,130.8,...,4.0,4.0,4.0,77.0,Spring,51.0,71.0,Winter,3.0,2.0


In [104]:
df[((np.abs(stats.zscore(df[numerical_cols], nan_policy='omit')) < 3) | (np.abs(stats.zscore(df[numerical_cols], nan_policy='omit')).isnull())).all(axis=1)]

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,ff8a2de4,Fall,13,0,Spring,60.0,Fall,16.362460,59.5,82.4,...,1.0,1.0,0.0,32.0,Winter,35.0,50.0,Fall,1.0,1.0
3956,ffa9794a,Winter,10,0,NaN,NaN,Spring,18.764678,53.5,76.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winter,0.0,NaN
3957,ffcd4dbd,Fall,11,0,Spring,68.0,Winter,21.441500,60.0,109.8,...,1.0,0.0,1.0,31.0,Winter,56.0,77.0,Fall,0.0,1.0
3958,ffed1dd5,Spring,13,0,Spring,70.0,Winter,12.235895,70.7,87.0,...,1.0,1.0,1.0,19.0,Spring,33.0,47.0,Spring,1.0,0.0


In [107]:
df[(np.abs(stats.zscore(df[numerical_cols], nan_policy='omit')) >= 3).any(axis=1)]

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
6,0038ba98,Fall,10,0,NaN,NaN,Fall,19.660760,55.00,84.6,...,4.0,1.0,0.0,20.0,Winter,27.0,40.0,Fall,3.0,0.0
28,015ba5b1,Spring,13,0,Fall,80.0,Fall,23.239092,60.20,119.8,...,1.0,1.0,1.0,50.0,Summer,36.0,51.0,Spring,3.0,2.0
51,02cebf33,Spring,12,1,Spring,77.0,Summer,19.751301,62.00,108.0,...,3.0,3.0,3.0,46.0,Summer,82.0,100.0,Spring,1.0,1.0
59,0380e709,Summer,17,0,NaN,NaN,Summer,24.446886,68.00,160.8,...,4.0,2.0,2.0,56.0,NaN,NaN,NaN,Summer,NaN,2.0
70,04370442,Fall,6,1,Summer,81.0,Spring,16.486185,50.75,60.4,...,5.0,5.0,5.0,74.0,Spring,36.0,51.0,Winter,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3905,fbb39bfe,Summer,16,0,NaN,NaN,Summer,20.980536,68.00,138.0,...,5.0,3.0,3.0,73.0,NaN,NaN,NaN,Summer,0.0,2.0
3922,fcd9a6bb,Spring,6,0,Summer,69.0,Spring,16.195145,44.00,44.6,...,1.0,0.0,0.0,14.0,Spring,38.0,54.0,Spring,0.0,0.0
3938,fe102a25,Summer,5,0,Winter,61.0,Summer,18.278557,43.50,49.2,...,3.0,0.0,0.0,34.0,NaN,NaN,NaN,Summer,3.0,1.0
3946,feb53635,Winter,14,0,Summer,70.0,Spring,19.036779,69.50,130.8,...,4.0,4.0,4.0,77.0,Spring,51.0,71.0,Winter,3.0,2.0


If we track outliers based on a $|zscore| >= 3$ metric, then that takes out 552 rows of the data!

But, the data is mostly skewed, so perhaps we can try something other than zscore for outlier detection.  
How about a quantile filter?

In [136]:
df[((df[numerical_cols] < df[numerical_cols].quantile(1)) | (df[numerical_cols].isnull())).all(axis=1)]

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
8,0069fbed,Summer,15,0,NaN,NaN,Spring,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,2.0,NaN
11,00abe655,Fall,11,0,Summer,66.0,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,10.0,Winter,42.0,59.0,Fall,0.0,0.0
14,00bd4359,Spring,12,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spring,2.0,NaN
19,00ebc35d,Winter,10,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winter,2.0,NaN
24,011e71c3,Summer,15,0,Fall,70.0,Summer,21.177784,62.0,115.8,...,2.0,NaN,2.0,30.0,Summer,34.0,49.0,Summer,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3942,fe6afaf9,Summer,11,0,Winter,60.0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,11.0,Summer,38.0,54.0,Fall,NaN,0.0
3943,fe7c87e2,Spring,13,0,Summer,NaN,Summer,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3951,ff18b749,Spring,7,0,NaN,NaN,Summer,14.768842,47.5,47.4,...,0.0,0.0,0.0,0.0,Summer,62.0,85.0,Spring,0.0,0.0
3958,ffed1dd5,Spring,13,0,Spring,70.0,Winter,12.235895,70.7,87.0,...,1.0,1.0,1.0,19.0,Spring,33.0,47.0,Spring,1.0,0.0


When we filter for rows that are all less than the 100% percentile (a trivial nonfilter), we only get 830 rows! What's the matter?

In [137]:
df[((df[numerical_cols] <= df[numerical_cols].quantile(1)) | (df[numerical_cols].isnull())).all(axis=1)]

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,ff8a2de4,Fall,13,0,Spring,60.0,Fall,16.362460,59.5,82.4,...,1.0,1.0,0.0,32.0,Winter,35.0,50.0,Fall,1.0,1.0
3956,ffa9794a,Winter,10,0,NaN,NaN,Spring,18.764678,53.5,76.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winter,0.0,NaN
3957,ffcd4dbd,Fall,11,0,Spring,68.0,Winter,21.441500,60.0,109.8,...,1.0,0.0,1.0,31.0,Winter,56.0,77.0,Fall,0.0,1.0
3958,ffed1dd5,Spring,13,0,Spring,70.0,Winter,12.235895,70.7,87.0,...,1.0,1.0,1.0,19.0,Spring,33.0,47.0,Spring,1.0,0.0


In [189]:
df[((df[true_numericals] < df[true_numericals].quantile(0.99)) | (df[true_numericals].isnull())).all(axis=1)]

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,ff8a2de4,Fall,13,0,Spring,60.0,Fall,16.362460,59.5,82.4,...,1.0,1.0,0.0,32.0,Winter,35.0,50.0,Fall,1.0,1.0
3956,ffa9794a,Winter,10,0,NaN,NaN,Spring,18.764678,53.5,76.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winter,0.0,NaN
3957,ffcd4dbd,Fall,11,0,Spring,68.0,Winter,21.441500,60.0,109.8,...,1.0,0.0,1.0,31.0,Winter,56.0,77.0,Fall,0.0,1.0
3958,ffed1dd5,Spring,13,0,Spring,70.0,Winter,12.235895,70.7,87.0,...,1.0,1.0,1.0,19.0,Spring,33.0,47.0,Spring,1.0,0.0


In [194]:
df[true_numericals].describe()

,Basic_Demos-Age,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,FGC-FGC_CU,...,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_C-PAQ_C_Total,PCIAT-PCIAT_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T
count,3960.000000,2421.000000,3015.000000,3027.000000,3015.000000,898.000000,2953.000000,2967.000000,2953.000000,1891.000000,...,1991.000000,1991.000000,1991.00000,1991.000000,1991.000000,1991.000000,1721.000000,2736.000000,2609.000000,2606.000000
mean,10.433586,65.454771,19.376812,55.946713,90.840060,27.278508,69.672536,81.597236,117.022689,13.826018,...,1.745354,33.173380,20.02299,67.301883,34.389466,53.998726,2.589550,27.896199,41.088923,57.763622
std,3.574648,22.341862,5.034191,7.473764,43.161374,5.567287,13.553022,13.665196,16.927687,11.649778,...,0.680635,56.272346,70.21561,108.705918,84.050607,129.362539,0.783937,20.338853,10.427433,13.196091
min,5.000000,25.000000,8.522436,33.000000,31.800000,18.000000,11.000000,27.000000,49.000000,1.000000,...,1.000000,14.489000,4.63581,23.620100,4.655730,20.589200,0.580000,0.000000,17.000000,38.000000
25%,8.000000,59.000000,15.890526,50.000000,58.200000,23.000000,61.000000,72.000000,107.000000,6.000000,...,1.000000,24.463500,12.98315,45.204100,21.141550,35.887000,2.020000,12.000000,33.000000,47.000000
50%,10.000000,65.000000,17.950925,55.000000,77.800000,26.000000,68.000000,81.000000,114.000000,10.000000,...,2.000000,28.855800,16.43880,56.996400,27.415100,44.987000,2.540000,26.000000,39.000000,55.000000
75%,13.000000,75.000000,21.588631,62.000000,114.300000,30.000000,76.000000,90.500000,125.000000,18.000000,...,2.000000,35.475700,22.16760,77.105650,38.179400,60.271050,3.160000,41.000000,46.000000,64.000000
max,22.000000,999.000000,59.132048,78.500000,315.000000,50.000000,179.000000,138.000000,203.000000,115.000000,...,3.000000,2457.910000,3108.17000,4683.710000,3607.690000,5690.910000,4.790000,93.000000,96.000000,100.000000


In [193]:
df[((df[true_numericals] > df[true_numericals].quantile(0)) | (df[true_numericals].isnull())).all(axis=1)]

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,001f3379,Spring,13,1,Winter,50.0,Summer,22.279952,59.5,112.2,...,1.0,2.0,1.0,34.0,Summer,40.0,56.0,Spring,0.0,1.0
6,0038ba98,Fall,10,0,NaN,NaN,Fall,19.660760,55.0,84.6,...,4.0,1.0,0.0,20.0,Winter,27.0,40.0,Fall,3.0,0.0
8,0069fbed,Summer,15,0,NaN,NaN,Spring,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3952,ff60112d,Summer,15,0,Spring,40.0,Winter,26.364710,70.5,186.4,...,1.0,1.0,1.0,35.0,Spring,66.0,90.0,Summer,3.0,1.0
3956,ffa9794a,Winter,10,0,NaN,NaN,Spring,18.764678,53.5,76.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winter,0.0,NaN
3957,ffcd4dbd,Fall,11,0,Spring,68.0,Winter,21.441500,60.0,109.8,...,1.0,0.0,1.0,31.0,Winter,56.0,77.0,Fall,0.0,1.0
3958,ffed1dd5,Spring,13,0,Spring,70.0,Winter,12.235895,70.7,87.0,...,1.0,1.0,1.0,19.0,Spring,33.0,47.0,Spring,1.0,0.0


It's because a lot of the rows 

In [124]:
df_filter_null = df

In [131]:
columns_df = df_filter_null.columns[1:]

In [132]:
# Make a copy of the original DataFrame
df_removed_outliers = df_filter_null.copy()

# Initialize a mask of True values for all rows (no outliers initially)
mask = pd.Series(True, index=df_filter_null.index)

# Iterate over each column in columns_df
for column in columns_df:
    # Check if the column is numeric
    if pd.api.types.is_numeric_dtype(df_filter_null[column]):
        # Calculate Q1, Q3, and IQR for the column
        q1 = df_filter_null[column].quantile(0.25)
        q3 = df_filter_null[column].quantile(0.75)
        IQR = q3 - q1
        lower_bound = q1 - 1.5 * IQR
        upper_bound = q3 + 1.5 * IQR

        # Update the mask: keep rows within bounds or where values are NaN
        mask &= ((df_filter_null[column] >= lower_bound) & (df_filter_null[column] <= upper_bound)) | df_filter_null[column].isna()

# Apply the mask to filter rows, retaining the original values
df_removed_outliers = df_filter_null[mask]


In [133]:
df_removed_outliers

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0069fbed,Summer,15,0,NaN,NaN,Spring,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,2.0,NaN
9,0083e397,Summer,19,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,0087dd65,Spring,11,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spring,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,ff8a2de4,Fall,13,0,Spring,60.0,Fall,16.362460,59.5,82.4,...,1.0,1.0,0.0,32.0,Winter,35.0,50.0,Fall,1.0,1.0
3956,ffa9794a,Winter,10,0,NaN,NaN,Spring,18.764678,53.5,76.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winter,0.0,NaN
3957,ffcd4dbd,Fall,11,0,Spring,68.0,Winter,21.441500,60.0,109.8,...,1.0,0.0,1.0,31.0,Winter,56.0,77.0,Fall,0.0,1.0
3958,ffed1dd5,Spring,13,0,Spring,70.0,Winter,12.235895,70.7,87.0,...,1.0,1.0,1.0,19.0,Spring,33.0,47.0,Spring,1.0,0.0


# Impute Na values with random values from the same column

In [ ]:
df_impute_random = df_removed_outliers.copy()
for column in columns_df:
  s = df_impute_random[column]

  # Step 1: Find the number of missing child heights for that gender.
  num_null = s.isna().sum()



  # Step 2: Sample num_null observed child heights for that gender.
  fill_values = np.random.choice(s.dropna(), num_null)

  # Step 3: Fill in missing
  s[s.isna()] = fill_values
  df_impute_random[column] = s





<ipython-input-27-f464eed2bdc1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[s.isna()] = fill_values
<ipython-input-27-f464eed2bdc1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[s.isna()] = fill_values
<ipython-input-27-f464eed2bdc1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[s.isna()] = fill_values
<ipython-input-27-f464eed2bdc1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [ ]:
df_impute_random



0

# Normalization

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df_min_max = df_impute_random.copy()


In [ ]:
numeric_columns = df_impute_random.select_dtypes(include=['number']).columns.tolist()


In [ ]:
scaler = MinMaxScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df_min_max[numeric_columns]), columns=numeric_columns)
df_min_max.loc[:, numeric_columns] = scaler.fit_transform(df_min_max[numeric_columns])

<ipython-input-31-091c912b04c0>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.26666667 0.86666667 0.66666667 ... 0.4        0.53333333 0.4       ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_min_max.loc[:, numeric_columns] = scaler.fit_transform(df_min_max[numeric_columns])


In [ ]:
df_min_max

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,PreInt_EduHx-Season,FGC-Season,Physical-Season,PreInt_EduHx-computerinternet_hoursday,Physical-Weight,Physical-Height,...,BIA-BIA_LST,BIA-BIA_LDM,BIA-BIA_ICW,BIA-BIA_Frame_num,BIA-BIA_Fat,BIA-BIA_FMI,BIA-BIA_FFMI,BIA-BIA_BMR,BIA-BIA_ECW,BIA-BIA_DEE
3,00115b9f,Winter,0.266667,0,Winter,Summer,Summer,0.000000,0.429022,0.490716,...,0.383296,0.398119,0.425619,0.5,0.364807,0.440460,0.341423,0.358956,0.375436,0.424623
4,0016bb22,Spring,0.866667,1,Fall,Summer,Summer,0.000000,0.600421,0.490716,...,0.117494,0.199338,0.590622,0.0,0.204643,0.318094,0.167600,0.473020,0.264961,0.295217
8,0069fbed,Summer,0.666667,0,Summer,Spring,Spring,0.666667,0.294427,0.278515,...,0.138186,0.488514,0.244213,0.5,0.179706,0.308451,0.248389,0.729283,0.361890,0.164531
9,0083e397,Summer,0.933333,1,Summer,Summer,Summer,0.000000,0.579390,0.649867,...,0.330978,0.579124,0.199993,0.5,0.405281,0.336218,0.316479,0.271066,0.426932,0.284811
10,0087dd65,Spring,0.400000,1,Spring,Winter,Winter,0.000000,0.334385,0.702918,...,0.502768,0.210892,0.363046,0.0,0.229161,0.463383,0.143596,0.127423,0.039867,0.449622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3953,ff6c2bb8,Fall,0.200000,0,Fall,Fall,Fall,0.666667,0.353312,0.397878,...,0.275742,0.364180,0.300133,0.0,0.303487,0.402998,0.235815,0.246043,0.247582,0.342955
3955,ff8a2de4,Fall,0.533333,0,Fall,Fall,Fall,0.333333,0.433228,0.583554,...,0.465669,0.440391,0.504961,0.0,0.258079,0.290390,0.339683,0.447552,0.483284,0.488697
3957,ffcd4dbd,Fall,0.400000,0,Fall,Winter,Winter,0.000000,0.577287,0.596817,...,0.522801,0.558705,0.521263,0.5,0.588592,0.629650,0.455921,0.502576,0.511758,0.465862
3958,ffed1dd5,Spring,0.533333,0,Spring,Spring,Winter,0.333333,0.457413,0.880637,...,0.689191,0.687739,0.718922,0.5,0.000000,0.045412,0.183764,0.691156,0.680414,0.947506


# One Hot encoding

In [98]:
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype


columns_df = df_impute_random.columns[1:]
categorical_columns = []
for column in columns_df:
  if is_string_dtype(df_impute_random[column]):
  #  print(column)
   # print(df[column].unique())
    categorical_columns.append(column)


# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
one_hot_encoded = encoder.fit_transform(df_impute_random[categorical_columns])

one_hot_df = pd.DataFrame(one_hot_encoded,
                          columns=encoder.get_feature_names_out(categorical_columns))

print(one_hot_df.shape)
print(df_impute_random.shape)

df_impute_random.reset_index(drop=True, inplace=True)
one_hot_df.reset_index(drop=True, inplace=True)
df_sklearn_encoded = pd.concat([df_impute_random.drop(categorical_columns, axis=1), one_hot_df], axis=1)



print(df_sklearn_encoded.shape)

#df_sklearn_encoded

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


df_sklearn_encoded.head()

(2236, 32)
(2236, 69)
(2236, 93)


,id,Basic_Demos-Age,Basic_Demos-Sex,PreInt_EduHx-computerinternet_hoursday,Physical-Weight,Physical-Height,Physical-BMI,Physical-HeartRate,Physical-Diastolic_BP,Physical-Systolic_BP,PCIAT-PCIAT_Total,sii,PCIAT-PCIAT_11,PCIAT-PCIAT_02,PCIAT-PCIAT_01,PCIAT-PCIAT_10,PCIAT-PCIAT_20,PCIAT-PCIAT_06,PCIAT-PCIAT_14,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_12,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_15,PCIAT-PCIAT_19,PCIAT-PCIAT_05,PCIAT-PCIAT_07,PCIAT-PCIAT_13,PCIAT-PCIAT_18,PCIAT-PCIAT_16,PCIAT-PCIAT_17,SDS-SDS_Total_Raw,SDS-SDS_Total_T,CGAS-CGAS_Score,FGC-FGC_TL,FGC-FGC_CU,FGC-FGC_PU,FGC-FGC_SRR,FGC-FGC_SRL,FGC-FGC_TL_Zone,FGC-FGC_CU_Zone,FGC-FGC_PU_Zone,FGC-FGC_SRR_Zone,FGC-FGC_SRL_Zone,BIA-BIA_TBW,BIA-BIA_FFM,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_SMM,BIA-BIA_LST,BIA-BIA_LDM,BIA-BIA_ICW,BIA-BIA_Frame_num,BIA-BIA_Fat,BIA-BIA_FMI,BIA-BIA_FFMI,BIA-BIA_BMR,BIA-BIA_ECW,BIA-BIA_DEE,Basic_Demos-Enroll_Season_Fall,Basic_Demos-Enroll_Season_Spring,Basic_Demos-Enroll_Season_Summer,Basic_Demos-Enroll_Season_Winter,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Winter,FGC-Season_Fall,FGC-Season_Spring,FGC-Season_Summer,FGC-Season_Winter,Physical-Season_Fall,Physical-Season_Spring,Physical-Season_Summer,Physical-Season_Winter,PCIAT-Season_Fall,PCIAT-Season_Spring,PCIAT-Season_Summer,PCIAT-Season_Winter,SDS-Season_Fall,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Winter,CGAS-Season_Fall,CGAS-Season_Spring,CGAS-Season_Summer,CGAS-Season_Winter,BIA-Season_Fall,BIA-Season_Spring,BIA-Season_Summer,BIA-Season_Winter
0,00115b9f,9,0,0.0,81.6,56.0,18.292347,97.0,60.0,117.0,44.0,1.0,3.0,2.0,4.0,2.0,1.0,1.0,0.0,4.0,0.0,0.0,3.0,2.0,0.0,4.0,5.0,0.0,3.0,3.0,3.0,4.0,31.0,45.0,71.0,7.0,18.0,5.0,7.0,7.0,1.0,1.0,0.0,0.0,0.0,45.9966,62.7757,3.0,3.84191,18.2943,26.4798,58.9338,16.7790,30.4041,2.0,18.82430,4.22033,14.0740,1131.430,15.59250,1923.44,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0016bb22,18,1,0.0,58.4,47.0,21.899516,118.0,65.0,107.0,0.0,0.0,2.0,0.0,4.0,2.0,1.0,1.0,0.0,3.0,0.0,0.0,2.0,1.0,4.0,1.0,1.0,0.0,0.0,2.0,0.0,3.0,29.0,58.0,75.0,12.0,0.0,10.0,10.0,10.0,1.0,1.0,0.0,1.0,1.0,38.0720,82.0602,4.0,5.31573,17.6237,25.6772,74.0944,16.3802,24.6740,2.0,24.84220,2.17012,13.3809,953.178,6.37437,2655.89,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0069fbed,15,0,2.0,132.2,44.0,16.560201,76.0,76.0,115.0,19.0,0.0,1.0,1.0,4.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,2.0,0.0,2.0,2.0,0.0,0.0,49.0,53.0,65.0,12.0,0.0,0.0,15.5,10.0,1.0,1.0,0.0,0.0,0.0,45.4306,59.8357,2.0,2.01781,23.5052,23.8034,40.1663,15.5909,25.3977,2.0,-5.90917,4.06521,14.7795,920.050,7.43282,1957.04,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0083e397,19,1,1.0,134.6,66.0,16.146408,75.0,64.0,98.0,12.0,0.0,0.0,2.0,2.0,1.0,0.0,1.0,1.0,3.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,2.0,2.0,0.0,2.0,27.0,59.0,50.0,10.0,10.0,0.0,10.0,11.0,1.0,1.0,1.0,0.0,1.0,42.2709,52.4305,2.0,4.45328,16.6825,26.7782,40.9378,15.3511,21.1202,1.0,5.76182,2.66389,14.1011,1282.910,6.56327,1554.21,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0087dd65,11,1,0.0,101.2,57.0,14.373669,88.0,72.0,101.0,7.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,60.0,42.0,61.0,7.0,0.0,1.0,5.0,9.0,1.0,0.0,0.0,1.0,1.0,34.1079,44.5290,3.0,2.97680,20.3814,17.9894,45.3493,11.0195,26.7165,2.0,13.50250,1.84164,13.8798,1041.230,10.62680,2201.87,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
